In [1]:
SURVEY_SUBMODULE_DIR = "./survey" # this notebook will create a folder "explanations" there
SAVE_PATH = "./results/user_study/selection.csv" # created by document_selection_user_study.ipynb

This notebook exports the user-study csv from `SAVE_PATH` to the UI. The UI uses a JSON file for each document and explainer-detector pair. The explanations are stored as HTML files. For LIME and SHAP, javascript is bundled, for Anchor, it must be imported due to bundle size. 

In [2]:
from detector_radford import DetectorRadford
from detector_detectgpt import DetectorDetectGPT
from detector_dummy import DetectorDummy
from detector_guo import DetectorGuo
from explainer_wrappers import LIME_Explainer, SHAP_Explainer, Anchor_Explainer

from tqdm import tqdm

In [3]:
explainer_classes = [LIME_Explainer, SHAP_Explainer, Anchor_Explainer]
detector_classes = [ DetectorGuo, DetectorRadford, DetectorDetectGPT]

In [4]:
import pandas as pd
import json
import os

In [5]:
df = pd.read_csv(SAVE_PATH)


In [6]:
df.groupby(["Detector"]).count()

,Unnamed: 0,Explainer,Documents Phases 1+3,Documents Phases 2+4,f(a),f(b),GT a,GT b,idx a,idx b,Spacy Similarity,Jaccard Similarity,Cosine Similarity tfidf,hash a,hash b
Detector,,,,,,,,,,,,,,,
DetectorDetectGPT,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18
DetectorGuo,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18
DetectorRadford,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18


In [7]:
if not os.path.exists(os.path.join(SURVEY_SUBMODULE_DIR)): 
    raise FileNotFoundError("Make sure you cloned the survey submodule")
if not os.path.exists(os.path.join(SURVEY_SUBMODULE_DIR, "explanations")): 
    os.makedirs(os.path.join(SURVEY_SUBMODULE_DIR, "explanations"))
if not os.path.exists(os.path.join(SURVEY_SUBMODULE_DIR, "explanations", "html")): 
    os.makedirs(os.path.join(SURVEY_SUBMODULE_DIR, "explanations", "html"))
if not os.path.exists(os.path.join(SURVEY_SUBMODULE_DIR, "explanations", "data")): 
    os.makedirs(os.path.join(SURVEY_SUBMODULE_DIR, "explanations", "data"))

While the explanations are cached, this will still take some time as `detector.predict_proba` must be called.

# Export for user-study

In [8]:

for detector_name, group in df.groupby("Detector"):
    if detector_name == "DetectorDetectGPT":
        detector = DetectorDetectGPT()
    if detector_name == "DetectorRadford":
        detector = DetectorRadford()
    if detector_name == "DetectorGuo":
        detector = DetectorGuo()

    documents_group = list(group.reset_index().iterrows()) # reset_index: count from 0 in each group
    
    for explainer_class in explainer_classes:
        explainer = explainer_class(detector)
        for index, row in tqdm(documents_group, "Exporting " + detector.__class__.__name__ + " " + explainer.__class__.__name__):
            a = row["Documents Phases 1+3"]
            b = row["Documents Phases 2+4"]
            # Phase 1 + 3: 
            path_explanation_html = os.path.join(SURVEY_SUBMODULE_DIR, "explanations", "html", explainer.get_hash(a)+".html")
            with open(path_explanation_html, "w", encoding="UTF-8") as text_file:
                text_file.write(explainer.get_HTML(a, bundle=False))
            path_explanation_json = os.path.join(SURVEY_SUBMODULE_DIR, "explanations", "data", explainer.get_hash(a)+".json")
            with open(path_explanation_json, "w", encoding="UTF-8") as text_file:
                explanation_data = { # do not include explanation
                    "document_nr": index,
                    "detector" : detector.__class__.__name__,
                    "explainer" : explainer.__class__.__name__,
                    "ground_truth" : row["GT a"],
                    "detector_label" : int(detector.predict_label([row["Documents Phases 1+3"]])[0]),
                    "detector_p_machine": float(detector.predict_proba([row["Documents Phases 1+3"]])[0][0]),
                    "detector_p_human": float(detector.predict_proba([row["Documents Phases 1+3"]])[0][1]),
                    "document": row["Documents Phases 1+3"],
                    "explanation_filename": explainer.get_hash(a),               
                }
                text_file.write(json.dumps(explanation_data))
            # Phase 2+4: Just string
            path_explanation_html = os.path.join(SURVEY_SUBMODULE_DIR, "explanations", "html", explainer.get_hash(b)+".html")
            #  display(HTML(explainer.get_HTML(b)))
            path_explanation_json = os.path.join(SURVEY_SUBMODULE_DIR, "explanations", "data", explainer.get_hash(b)+".json")
            with open(path_explanation_json, "w", encoding="UTF-8") as text_file:
                explanation_data = { # do not include explanation
                    "document_nr": index,
                    "detector" : detector.__class__.__name__,
                    "explainer" : explainer.__class__.__name__,
                    "document": row["Documents Phases 2+4"]                
                }
                text_file.write(json.dumps(explanation_data))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Exporting DetectorDetectGPT Anchor_Explainer: 100%|██████████| 18/18 [00:51<00:00,  2.89s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Exporting DetectorRadford Anchor_Explainer: 100%|██████████| 18/18 [00:02<00:00,  7.57it/s]


# Export as HTML for review

## One file for the selecting combinations

In [9]:
out = "<html><body>"
out += "<h1>Pairs as generated in the notebook</h1>"

for (detector_name, explainer_name), group in df.groupby(["Detector", "Explainer"]):
    out += "<h2>{}</h2>".format(detector_name)
    if detector_name == "DetectorDetectGPT":
        detector = DetectorDetectGPT()
    if detector_name == "DetectorRadford":
        detector = DetectorRadford()
    if detector_name == "DetectorGuo":
        detector = DetectorGuo()

    if explainer_name == "LIME_Explainer":
        explainer = LIME_Explainer(detector)
    if explainer_name == "SHAP_Explainer":
        explainer = SHAP_Explainer(detector)
    if explainer_name == "Anchor_Explainer":
        explainer = Anchor_Explainer(detector)
    documents_group = list(group.reset_index().iterrows()) # reset_index: count from 0 in each group
    

    for index, row in tqdm(documents_group, "Exporting " + detector.__class__.__name__ + " " + explainer.__class__.__name__):
        a = row["Documents Phases 1+3"]
        b = row["Documents Phases 2+4"]
        template = """
        Document: {document_nr}<br/>
        Detector: {detector}<br/>
        Explainer: {explainer}<br/>
        Ground Truth: {ground_truth}<br/>
        f(x): {detector_label}
        """
        # Phase 1 + 3: 
        out += "<h3>Pair {}</h3>".format(index)

        explanation_data = { 
            "document_nr": row["idx a"],
            "detector" : detector.__class__.__name__,
            "explainer" : explainer.__class__.__name__,
            "ground_truth" : row["GT a"],
            "detector_label" : int(detector.predict_label([row["Documents Phases 1+3"]])[0]),
        }
        out+= template.format(**explanation_data)
        out += explainer.get_HTML(a, bundle=True)
       

        explanation_data = { 
            "document_nr": row["idx b"],
            "detector" : detector.__class__.__name__,
            "explainer" : explainer.__class__.__name__,
            "ground_truth" : row["GT b"],
            "detector_label" : int(detector.predict_label([row["Documents Phases 2+4"]])[0]),           
        }
        out+= template.format(**explanation_data)
        out += explainer.get_HTML(b, bundle=True)
        out +="<hr/>"
out +="</body></html>"
with open("rendered_datasets_user_study/dataset.html", "w", encoding="UTF-8") as text_file:
    text_file.write(out)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Exporting DetectorDetectGPT Anchor_Explainer: 100%|██████████| 6/6 [00:12<00:00,  2.09s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Exporting DetectorDetectGPT LIME_Explainer: 100%|██████████| 6/6 [00:12<00:00,  2.09s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Exporting DetectorDetectGPT SHAP_Explainer: 100%|██████████| 6/6 [00:14<00:00,  2.33s/it]
Special tokens have 

## And one per group

In [10]:


for detector_name, group in df.groupby("Detector"):
   
    if detector_name == "DetectorDetectGPT":
        detector = DetectorDetectGPT()
    if detector_name == "DetectorRadford":
        detector = DetectorRadford()
    if detector_name == "DetectorGuo":
        detector = DetectorGuo()


    documents_group = list(group.reset_index().iterrows()) # reset_index: count from 0 in each group
    for explainer_class in explainer_classes:
        explainer = explainer_class(detector)
        out = "<html><body>"
        out += "<h1>Pairs as displayed to this group</h1>"
        out += "<p>Users only see the first explanation of each pair.</p>"
        for index, row in tqdm(documents_group, "Exporting " + detector.__class__.__name__ + " " + explainer.__class__.__name__):
            a = row["Documents Phases 1+3"]
            b = row["Documents Phases 2+4"]
            template = """
            Document: {document_nr}<br/>
            Displayed Explainer: {explainer}<br/>
            Selecting Explainer: {explainer_s}<br/>
            Ground Truth: {ground_truth}<br/>
            f(x): {detector_label}
            """
            # Phase 1 + 3: 
            out += "<h3>Pair {}</h3>".format(index)

            explanation_data = { 
                "document_nr": row["idx a"],
                "explainer" : explainer.__class__.__name__,
                "explainer_s" : row["Explainer"],
                "ground_truth" : row["GT a"],
                "detector_label" : int(detector.predict_label([row["Documents Phases 1+3"]])[0]),
            }
            out+= template.format(**explanation_data)
            out += explainer.get_HTML(a, bundle=True)
        

            explanation_data = { 
                "document_nr": row["idx b"],
                "explainer" : explainer.__class__.__name__,
                "explainer_s" : row["Explainer"],
                "ground_truth" : row["GT b"],
                "detector_label" : int(detector.predict_label([row["Documents Phases 2+4"]])[0]),           
            }
            out+= template.format(**explanation_data)
            out += explainer.get_HTML(b, bundle=True)
            out +="<hr/>"
        out +="</body></html>"
        with open("rendered_datasets_user_study/groups/{}-{}.html".format(detector_name, explainer.__class__.__name__), "w", encoding="UTF-8") as text_file:
            text_file.write(out)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Exporting DetectorDetectGPT Anchor_Explainer: 100%|██████████| 18/18 [00:40<00:00,  2.26s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Exporting DetectorRadford Anchor_Explainer: 100%|██████████| 18/18 [00:05<00:00,  3.41it/s]


In [3]:
from IPython.core.display import HTML


In [5]:
SHAP_Explainer(DetectorDummy(human_watermark="example", machine_watermark="an")).get_HTML("This is an example.")

'<svg width="100%" height="80px"><line x1="0" y1="33" x2="100%" y2="33" style="stroke:rgb(150,150,150);stroke-width:1" /><line x1="49.99999975%" y1="33" x2="49.99999975%" y2="37" style="stroke:rgb(150,150,150);stroke-width:1" /><text x="49.99999975%" y="27" font-size="12px" fill="rgb(120,120,120)" dominant-baseline="bottom" text-anchor="middle">0.5</text><line x1="34.999999825%" y1="33" x2="34.999999825%" y2="37" style="stroke:rgb(150,150,150);stroke-width:1" /><text x="34.999999825%" y="27" font-size="12px" fill="rgb(120,120,120)" dominant-baseline="bottom" text-anchor="middle">0.2</text><line x1="19.999999900000002%" y1="33" x2="19.999999900000002%" y2="37" style="stroke:rgb(150,150,150);stroke-width:1" /><text x="19.999999900000002%" y="27" font-size="12px" fill="rgb(120,120,120)" dominant-baseline="bottom" text-anchor="middle">-0.1</text><line x1="64.999999675%" y1="33" x2="64.999999675%" y2="37" style="stroke:rgb(150,150,150);stroke-width:1" /><text x="64.999999675%" y="27" font-s